**【注意】marker_labels是固定的，因此其数量也是固定的，marker_data随机数产生；gender固定为b'male'；frame_length固定为30；dmpls和betas初始化为0；**

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from scipy.spatial.transform import Rotation

def convert_folder_to_amass_npz(folder_path, output_folder, mapping_table):
    os.makedirs(output_folder, exist_ok=True)
    npy_files = [os.path.join(folder_path, root, file) for root, dirs, files in os.walk(folder_path) for file in files if file.endswith(".npy")]

    for npy_file_path in npy_files:
        npz_file_path = os.path.join(output_folder, os.path.relpath(npy_file_path, folder_path).replace(".npy", ".npz"))

        convert_to_amass_npz(npy_file_path, npz_file_path, mapping_table)

def convert_to_amass_npz(npy_path, npz_path, mapping_table):
    motion = np.load(npy_path)
    motion = torch.tensor(motion).float()

    motion_parms = {
        'root_orient': motion[:, :3],
        'pose_body': motion[:, 3:3+63],
        'pose_hand': motion[:, 66:66+90],
        'trans': motion[:, 309:309+3],
        'betas': motion[:, 312:],
    }

    frame_length = motion.shape[0]
    gender = b'male'
    mocap_framerate = 30
    betas = np.zeros(16)
    dmpls = np.zeros((frame_length, 8))
    poses = np.zeros((frame_length, 156))
    trans = np.zeros((frame_length, 3))

    for frame_index in range(frame_length):
        pelvis_position = motion_parms['trans'][frame_index].numpy() * 0.01

        joint_rotations = {}
        for order, joint_data in mapping_table.iterrows():
            joint_name = joint_data.name
            if joint_name not in motion_parms:
                continue

            rotation_angles = motion_parms[joint_name][frame_index].numpy()
            joint_rotations[order] = Rotation.from_euler('xyz', rotation_angles, degrees=True).as_rotvec()

        poses[frame_index, :63] = motion_parms['pose_body'][frame_index].numpy()
        poses[frame_index, 63:153] = motion_parms['pose_hand'][frame_index].numpy()

        for order in joint_rotations.keys():
            poses[frame_index, 153 + order * 3:156 + order * 3] = joint_rotations[order]

        trans[frame_index, :] = pelvis_position

    marker_labels = [b'RBSH', b'LFTHI', b'RBHD', b'RPSI', b'LELBIN', b'LBSH', b'RTHMB', b'RFRM2IN', b'LFIN', 
                     b'RFIN', b'LTHMB', b'LOWR', b'LFRM2', b'LELB', b'LBCEP', b'LRSTBEEF', b'LSHO', b'LKNE', 
                     b'LBUST', b'LTOE', b'LMT1', b'LFTHIIN', b'MBLLY', b'STRN', b'RKNI', b'RCHEECK', b'RFHD', 
                     b'RANK', b'RFTHIIN', b'RKNE', b'RFTHI', b'RBUSTLO', b'RMT5', b'RSHO', b'RRSTBEEF', b'RBCEP', 
                     b'RELB', b'RFRM2', b'ROWR', b'RTHI', b'RTIB', b'RASI', b'LHEE', b'LASI', b'LNWST', b'RBUM', 
                     b'LPSI', b'RHEE', b'LTIB', b'LCHEECK', b'LFHD', b'LANK', b'RBAK', b'C7', b'LBHD', b'LTHI', 
                     b'LBUM', b'LBAK', b'T8', b'RELBIN', b'CLAV', b'RTOE', b'LMT5', b'LKNI', b'RMT1']
    marker_data = np.random.rand(frame_length, len(marker_labels), 3)

    os.makedirs(os.path.dirname(npz_path), exist_ok=True)
    np.savez(npz_path, poses=poses, gender=gender, mocap_framerate=mocap_framerate,
             betas=betas, marker_data=marker_data, dmpls=dmpls, marker_labels=marker_labels, trans=trans)

npy_folder_path = 'Motion-X\motion_data'
npz_output_folder = 'Motion-X\motion_data\\npz'

keys_mapping_table = pd.read_csv("json2amass_by_syan/keys_mapping.csv")
keys_mapping_table.set_index('fbx_name', inplace=True)

convert_folder_to_amass_npz(npy_folder_path, npz_output_folder, keys_mapping_table)